# **Zuber Ride-Share Analysis, Chicago 2017**

## **Introduction**
- The objective of this project is to develop recommendations for optimizing Zuber's rideshare presence in the Chicago area, and potentially extrapolate that data to enter additional metropolitan markets.
- Competitor rideshare data and publicly available information pertaining to weather forecasts for the focal time period will be analyzed to draw insights that will infer customer preferences and direct the recommendations.

## **Approach**
The dataset will be analyzed to understand the following:
- Top 10 Neighbordhoods Relative to Drop-Offs
- Competitor Pareto Analysis (Total Rides per Company)
- Hypothesis Test: Influence of Rainy Weather on Saturday Trips to the Airport

The project will consist of the following stages:
1. Data Preparation
2. Data Analysis
3. Hypothesis Testing
4. Conclusion

## **Initialization**
Importing all relevant libraries and loading in the datasets.

In [5]:
import pandas as pd
import numpy as np
import math as mt
import random as rd
import plotly.express as px
from matplotlib import pyplot as plt
from scipy import stats as st

In [6]:
# Load the datasets
company_data = pd.read_csv('../data/moved_project_sql_result_01.csv')
dropoff_data = pd.read_csv('../data/moved_project_sql_result_04.csv')
airport_ride_data = pd.read_csv('../data/moved_project_sql_result_07.csv')

# Display first few rews to verify datasets
print(company_data.head())
print()
print(dropoff_data.head())
print()
print(airport_ride_data.head())

                      company_name  trips_amount
0                        Flash Cab         19558
1        Taxi Affiliation Services         11422
2                 Medallion Leasin         10367
3                       Yellow Cab          9888
4  Taxi Affiliation Service Yellow          9299

  dropoff_location_name  average_trips
0                  Loop   10727.466667
1           River North    9523.666667
2         Streeterville    6664.666667
3             West Loop    5163.666667
4                O'Hare    2546.900000

              start_ts weather_conditions  duration_seconds
0  2017-11-25 16:00:00               Good            2410.0
1  2017-11-25 14:00:00               Good            1920.0
2  2017-11-25 12:00:00               Good            1543.0
3  2017-11-04 10:00:00               Good            2512.0
4  2017-11-11 07:00:00               Good            1440.0


## **Data Preparation**
For each dataset, the following items shall be performed where necessary:
- Replace column names
- Convert data to required types
- Address missing and/or duplicate values as-necessary

### **Data Preparation: `company_data`**

In [7]:
# Print the general/summary information about the dataset
company_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [8]:
# Verify the number of duplicated rows and remove if necessary
company_data.duplicated().sum()

np.int64(0)

In [9]:
# Review number of missing values
print(company_data.isna().sum())

company_name    0
trips_amount    0
dtype: int64


- Column names are in correct format, all lowercase/snake-case.
- All column data types are appropriate for analysis.
- No duplicate values were detected.
- No missing values were detected.
- This dataset is ready for analysis. 

### **Data Preparation: `dropoff_data`**

In [10]:
# Print the general/summary information about the dataset
dropoff_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB


In [11]:
# Verify the number of duplicated rows and remove if necessary
dropoff_data.duplicated().sum()

np.int64(0)

In [12]:
# Review number of missing values
print(dropoff_data.isna().sum())

dropoff_location_name    0
average_trips            0
dtype: int64


- Column names are in correct format, all lowercase/snake-case.
- All column data types are appropriate for analysis.
- No duplicate values were detected.
- No missing values were detected.
- This dataset is ready for analysis. 

### **Data Preparation: `airport_ride_data`**

In [13]:
# Print the general/summary information about the dataset
airport_ride_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_seconds    1068 non-null   float64
dtypes: float64(1), object(2)
memory usage: 25.2+ KB


In [14]:
# Verify the number of duplicated rows and remove if necessary
print(airport_ride_data.duplicated().sum())

print()

# Assess nature of duplicated rows
airport_dupes = airport_ride_data[airport_ride_data.duplicated()]
print(airport_dupes)

# Review percentage of duplicate values to determine impact to dataset
percent_airport_dupes = (airport_ride_data.duplicated().sum()/len(airport_ride_data)) * 100
print()
print(f"{percent_airport_dupes:.2f} % of the rows in the airport_ride_data file are duplicated")

197

                 start_ts weather_conditions  duration_seconds
62    2017-11-11 06:00:00               Good            1260.0
74    2017-11-11 08:00:00               Good            1380.0
76    2017-11-04 09:00:00               Good            1380.0
117   2017-11-11 07:00:00               Good            1380.0
119   2017-11-04 14:00:00               Good            3300.0
...                   ...                ...               ...
1054  2017-11-11 07:00:00               Good            1380.0
1058  2017-11-25 12:00:00               Good            1440.0
1062  2017-11-11 06:00:00               Good            1500.0
1065  2017-11-11 13:00:00               Good            2100.0
1066  2017-11-11 08:00:00               Good            1380.0

[197 rows x 3 columns]

18.45 % of the rows in the airport_ride_data file are duplicated


In [15]:
#Review number of missing values
print(airport_ride_data.isna().sum())

start_ts              0
weather_conditions    0
duration_seconds      0
dtype: int64


- Column names are in correct format, all lowercase/snake-case.
- All column data types are appropriate for analysis.
- 197 duplicate values were detected.
    - These duplicate values shall be included in the dataset because the `start_ts` column is only precise to the hour, and there may have been several rides that occurred within the same interval that took the same time to complete.
    - As nearly 20% of the data is impacted by the duplicates and the `start_ts` column lacks the precision to wholesale exclude these rows, they shall not be removed.
- No missing values were detected.
- This dataset is ready for analysis. 

## **Data Analysis**

In [16]:
# Identify the top 10 neighborhoods in terms of trips
top_10_nbs = dropoff_data.head(10)
top_10_nbs

,dropoff_location_name,average_trips
0,Loop,10727.466667
1,River North,9523.666667
2,Streeterville,6664.666667
3,West Loop,5163.666667
4,O'Hare,2546.900000
5,Lake View,2420.966667
6,Grant Park,2068.533333
7,Museum Campus,1510.000000
8,Gold Coast,1364.233333
9,Sheffield & DePaul,1259.766667


In [17]:
# Visualize Top 10 Neighborhood ins Terms of Trips
top_10_nbs = px.bar(top_10_nbs,
                   x='dropoff_location_name',
                   y='average_trips',
                   color='dropoff_location_name',
                   title='Top 10 Destinations by Trip Count',
                   template='plotly_dark',
                   color_discrete_sequence=px.colors.qualitative.Dark2)

top_10_nbs.update_layout(
    yaxis_title='Average Number of Trips',
    xaxis_title='Dropoff Neighborhood',
    legend_title='Dropoff Neighborhood')

top_10_nbs.show()

__CONCLUSIONS:__
- There are four standout neighborhood in the top 10 for average number of trips, including:
    - Loop
    - River North
    - Streeterville
    - West Loop
- Concentrations of Zuber presence should prioritize these neighborhoods above other potential destinations in Chicago, if resources are constrained to the critical few.
- It may be worth incentivizing potential drivers for Zuber with greater ride payout percentages in these areas to compete more aggressively in the hot zones.
- An alternative strategy would be to subsidize/discount trips for riders during peak travel seasons and undercut the competition to capitalize on the seasonal increase in riders.

In [18]:
# Visualize number of rides for each taxi company from 11/15 - 11/16 2017
competitor_data = px.bar(company_data,
                   x='company_name',
                   y='trips_amount',
                   color='company_name',
                   title='Competitor Trip Analysis (November 15 - 16, 2017)',
                   template='plotly_dark',
                   color_discrete_sequence=px.colors.qualitative.Dark2)

competitor_data.update_layout(
    yaxis_title='Number of Trips',
    xaxis_title='Company',
    legend_title='Company')

competitor_data.show()

There were many active taxi companies operating on that particular weekend - we will re-run the analysis on the Top 10 for a more clean visualization.

In [19]:
# Identify the top 10 neighborhoods in terms of trips
top_competitors = company_data.head(10)
top_competitors

,company_name,trips_amount
0,Flash Cab,19558
1,Taxi Affiliation Services,11422
2,Medallion Leasin,10367
3,Yellow Cab,9888
4,Taxi Affiliation Service Yellow,9299
5,Chicago Carriage Cab Corp,9181
6,City Service,8448
7,Sun Taxi,7701
8,Star North Management LLC,7455
9,Blue Ribbon Taxi Association Inc.,5953


In [20]:
# Visualize number of rides for each top taxi company from 11/15 - 11/16 2017
top_competitor_data = px.bar(top_competitors,
                   x='company_name',
                   y='trips_amount',
                   color='company_name',
                   title='Top 10 Competitors Trip Analysis (November 15 - 16, 2017)',
                   template='plotly_dark',
                   color_discrete_sequence=px.colors.qualitative.Dark2)

top_competitor_data.update_layout(
    yaxis_title='Number of Trips',
    xaxis_title='Company',
    legend_title='Company')

top_competitor_data.show()

__CONCLUSIONS:__
- Flash Cab is significantly more competitive than the remainder of the Top 10, with approximately 19.5k trips completed on the Nov. 15th weekend.
    - Its nearest competitors, Taxi Affiliation Services, Medallion Leasing, and Yellow Cab trail significantly, with trip counts between 9.5k and 11.4k amongst each of them.
- Zuber should study Flash Cab's marketing decisions closely and work to actively beat them on competitive pricing and neighborhood presence and steal market share.

## **Hypothesis Testing**

The following code will perform hypothesis testing to determine whether rainy weather has an impact on the average duration of the rides on Saturday trips from the Loop neighborhood to the O'Hare International Airport.

**KEY INFORMATION**:
- __H<sub>0</sub>__ = The average duration of rides on rainy Saturday trips from the Loop neighborhood to the O'Hare International Airport do not differ significantly from Saturday rides where the weather is not rainy.
- __H<sub>1</sub>__ = The average duration of rides on rainy Saturday trips from the Loop neighborhood to the O'Hare International Airport differs significantly from Saturday rides where the weather is not rainy.
- __&alpha;__ = 0.05: Selected as 0.05 is standard for most business analysis hypothesis testing.
- __`st.ttest_ind()`__: Selected this statistical test as the hypothesis is concerned with asssessing the equality of two population means.

In [21]:
# Test the hypotheses
rainy_saturdays = airport_ride_data[airport_ride_data['weather_conditions'] == 'Bad']['duration_seconds']
not_rainy_saturdays = airport_ride_data[airport_ride_data['weather_conditions'] == 'Good']['duration_seconds']

alpha = 0.05 #Standard acceptable alpha for business research

test_results = st.ttest_ind(rainy_saturdays, not_rainy_saturdays, equal_var=False)

print('p-value: ', test_results.pvalue)

print()

if test_results.pvalue < alpha:
    print ('We reject the null hypothesis')
else:
    print('We can\'t reject the null hypothesis')

p-value:  6.738994326108734e-12

We reject the null hypothesis


__Conclusions & Commentary__:
- The initial prompt "average duration of rides on rainy Saturday trips from the Loop neighborhood to the O'Hare International Airport do not differ significantly from Saturday rides where the weather is not rainy" follows the logic for describing a null hypothesis, which is a statement of confirmation (i.e., The mean of the population in question = A). 
    - Therefore, the opposite would be the alternative hypothesis, whereby the average duration of rides... are not the same. 
- An alpha value of 0.05 was selected to compare the p-value againts as this is a standard acceptable value used for business research.
- The hypothesis test rejected the null hypothesis on the grounds that the p-value calculated in the test was less than the alpha value.
- The test supports the alternative hypothesis, in that the average duration of Saturday rides from Loop to the airport differs signficiantly on weekends where the weekend is rainy vs. not-rainy.
    - More customers may be opting to take a taxi on the rainy days from further away to avoid public transportation where the exposed portions of the commute would be unpleasant to traverse (oudoor bus/train stops).
    - Additionally, more customers may also call for a cab from further away to avoid having to drive in more dangerous weather themselves.

## **Concusion**
__Project Objectives:__
- Develop recommendations for optimizing Zuber's rideshare presence in the Chicago area, and potentially extrapolate that data to enter additional metropolitan markets.
- Load, cleanse, transform, and visualize datasets containing competitor trip dropoff and trip data, as well as publicly available online weather data.'
- Provide insights to client, Zuber, to inform them of passengers preferences and external factor influence on rides and drive effective marketing strategy.

__Results Summary:__
- There are four standout neighborhood in the top 10 for average number of trips, including:
    - Loop
    - River North
    - Streeterville
    - West Loop
- Concentrations of Zuber presence should prioritize these neighborhoods above other potential destinations in Chicago, if resources are constrained to the critical few.
- Flash Cab is significantly more competitive than the remainder of the Top 10 Competitors, with approximately 19.5k trips completed on the Nov. 15th weekend.
- Hypothesis testing results found that the average duration of Saturday rides from the Loop neighborhood to the O'Hare International Airport differs signficiantly on weekends where the weekend was rainy vs. not-rainy.

__Next Steps & Recommendations:__
- Explore incentivizing potential drivers for Zuber with greater ride payout percentages in these areas to compete more aggressively in the hot zones.
    - Alternatively or additionally, subsidize/discount trips for riders during peak travel seasons and undercut the competition to capitalize on the seasonal increase in riders.'
- Study Flash Cab's marketing decisions closely and work to actively beat them on competitive pricing and neighborhood presence and steal market share.
- More customers may be opting to take a taxi on the rainy days from further away to avoid public transportation where the exposed portions of the commute would be unpleasant to traverse (oudoor bus/train stops). Additionally, more customers may also call for a cab from further away to avoid having to drive in more dangerous weather themselves.
    - Double-down on the above recommendations  for Zuber presence and financial incentives in the popular neighborhoods identified above on the rainier days to capitalize on customer preference during inclement weather periods.
    - It may be worth investing/incentivising Zuber drivers that drive vehicles better suited for driving in inclement weather and/or are perceived to be safer by riders, such as larger vehicles (trucks/SUVs/vans) with features such all-wheel drive (AWD).
    - There may an opportunity here offer this as an option for a slight surcharge to customers to ensure that this type of vehicles is what picks them up for their ride.